In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Step 1 : Importing and Understanding Data

In [ ]:
#importing libraries and shuffling the data
df = pd.read_csv('/kaggle/input/autompg-dataset/auto-mpg.csv')
df = df.sample(frac = 1,random_state = 3)
df.head()

In [ ]:
#Finding the unqiue values
df['origin'].unique()

#here we see there is '?' here.

In [ ]:
#To check the shape of Dataframe
df.shape

In [ ]:
#To check the data-type of the dataframe
df.info()

In [ ]:
#To check descriptive statistics of data frame
df.describe()

# Exploratory Data Analysis

## Missing values

In [ ]:
df.isna().sum()*100/398

No mising values

# Cleaning The Data

# Removing ? from hp and converting its data type to INT

In [ ]:
#Removing ?, replacing with mean value and conver the datatype to integer
df['horsepower'] = df['horsepower'].replace('?',np.nan)
df['horsepower'] = df['horsepower'].astype(float)
df['horsepower'] = df['horsepower'].replace(np.nan,df['horsepower'].mean())
df['horsepower'] = df['horsepower'].astype(int)


? removed and converted into int type

# Univariate Analysis

# Bar Plot for car name

In [ ]:
df['car name'].value_counts().nlargest(15).plot(kind='bar', figsize=(15,5))
plt.title("Number of vehicles by car brand name")
plt.ylabel('Number of vehicles')
plt.xlabel('car name');

**Ford Pinto is the car name which has most number of vehicles and Ford Marverick holds the second poisition**

# Histogram for mpg


In [ ]:
df['mpg'].hist(bins = 5,color = 'red')
plt.title("Miles per gallon of vehicle")
plt.ylabel('Number of vehicles')
plt.xlabel('Miles per gallon');

# Histogram for cylinders

In [ ]:
df['cylinders'].hist(bins = 5,color = 'red')
plt.title("Number of Cylinders in a vehicle")
plt.ylabel('Number of vehicles')
plt.xlabel('Numer of Cylinder');

**Maximum number of cars are having cylinder between 4 to 5**

# Histogram for displacement

In [ ]:
df['displacement'].hist(bins = 5,color = 'red')
plt.title("Displacement of the vehicle")
plt.ylabel('Number of vehicles')
plt.xlabel('Displacement');

**Maximum number of cars are having dispalcement between 70 to 140**

# Histogram for weight

In [ ]:
df['weight'].hist(bins = 5,color = 'red')
plt.title("Weight of vehicle")
plt.ylabel('Number of vehicles')
plt.xlabel('Weight');

**Max weight of car are between 1600 to 3000  **

# Histogram for acceleration

In [ ]:
df['acceleration'].hist(bins = 5,color = 'red')
plt.title("Acceleration of vehicle")
plt.ylabel('Number of vehicles')
plt.xlabel('Acceleration');

**General acceleration of cars are between 15.0 to 18.0**

# Histogram for model year

In [ ]:
df['model year'].hist(bins = 5,color = 'red')
plt.title("Model year of vehicle")
plt.ylabel('Number of vehicles')
plt.xlabel('Year');

**We can say that most of the cars were made during the year 1970-72, 1975-77 and 1980-82 **

In [ ]:
df[['mpg','cylinders','displacement','weight','acceleration','model year','origin']].hist(figsize=(10,8),bins=6,color='Y')
plt.tight_layout()
plt.show()

# Bivariate Analysis

# Box plot btw mpg and weight

In [ ]:
plt.rcParams['figure.figsize']=(23,10)
ax = sns.boxplot(x="mpg", y="weight", data=df)

**We can see that as the weight of the car increases, the mileage decreases which is obvious because they are inversely proportional to each other**

 # Box plot btw mpg and displacement

In [ ]:
plt.rcParams['figure.figsize']=(23,10)
ax = sns.boxplot(x="mpg", y="displacement", data=df)

**We see that as the displacement increases,mileage decreases.**

# Box plot btw horsepower and weight

In [ ]:
plt.rcParams['figure.figsize']=(23,10)
ax = sns.boxplot(x="horsepower", y="weight", data=df)

**We can clearly see that as the weight increases,the horsepower increases.Otherwise car with heavy weight won't run without greater horsepower. So they are directly proportional**

# Scatter plot for weight of car and acceleration

In [ ]:
sns.scatterplot(x = 'weight',y = 'acceleration',data = df)

**It is showing that lesser the weight of car,more is the acceleration.**

# Correlation Analysis

In [ ]:
corr = df.corr()
sns.set_context("notebook", font_scale=1.0, rc={"lines.linewidth": 2.5})
plt.figure(figsize=(13,7))
a = sns.heatmap(corr, annot=True, fmt='.2f')
rotx = a.set_xticklabels(a.get_xticklabels(), rotation=90)
roty = a.set_yticklabels(a.get_yticklabels(), rotation=30)

# Building Statistical Model

In [ ]:
#import library
import statsmodels.api as sm

In [ ]:
#drop car name
df = df.drop('car name',axis = 1)

In [ ]:
#Taking target variable and adding constant
y = df['mpg']
X = df.drop('mpg',axis = 1)
Xc = sm.add_constant(X)

In [ ]:
#importing vif
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [ ]:
pd.DataFrame([vif(Xc.values,i) for i in range(Xc.shape[1])],index = Xc.columns,columns=['VIF'])

In [ ]:
#Fitting model
model = sm.OLS(y,Xc).fit()
model.summary()

Here wee see that cylindes,horsepower,acceleration all three are not significant they are above 0.05 so we will drop it.

In [ ]:
#Dropping cylinders,horsepower,acceleration and fiting into model
Xc = Xc.drop(['cylinders','horsepower','acceleration'],axis = 1)
model = sm.OLS(y,Xc).fit()
model.summary()

# Test of Assumptions

# 1. Test of Normality

In [ ]:
#importing library and creating distplot
from scipy.stats import norm
norm.fit(model.resid)
sns.distplot(model.resid,fit = norm)

here wee se its almost normal.

In [ ]:
#importing library and creating probplot
import scipy.stats as st
st.probplot(model.resid,plot = plt)
plt.show()

Ho: Data is normal

H1: Data is not normal

In [ ]:
#Jarque Bera Test to check normality
st.jarque_bera(model.resid)

p-value very close to zero so reject null hypothesis. Hence JB Test indicate that residuals are not normal.

In [ ]:
#Applying Transformation
ly = np.log(y)

In [ ]:
#Building model
model = sm.OLS(ly,Xc).fit()
model.summary()

In [ ]:
#Again creating distplot after applying log
from scipy.stats import norm
norm.fit(model.resid)
sns.distplot(model.resid,fit = norm)

In [ ]:
#Again creating probplot after applying log
import scipy.stats as st
st.probplot(model.resid,plot = plt)
plt.show()

In [ ]:
#Again performing JB Test
st.jarque_bera(model.resid)

p-value again closed to zero, so reject Ho. Hence Residuals are not normal

# 2.Test of Homoscedasticity

In [ ]:
#Splitting X and y and adding constant
y = df['mpg']
X = df.drop('mpg',axis = 1)
Xc = sm.add_constant(X)

In [ ]:
model = sm.OLS(y,Xc).fit()
y_pred = model.predict(Xc)
resids = model.resid

sns.regplot(x = y_pred,y = resids,lowess=True,line_kws={'color':'red'})

In [ ]:
#Goldfeld Test for Checking Homoscedasticity
import statsmodels.stats.api as sms
from statsmodels.compat import lzip

name = ['F-Statistics','p-value']
test = sms.het_goldfeldquandt(model.resid,model.model.exog)
print(lzip(name,test))

Ho: Variance of residuals is constant across the range of data.
H1: Variance of residuals is not constant across the range of data.
    
Since p-value is more than 0.05 we will accept  Ho to conclude that residuals is constatnt across the range of data.

# 3.Test of Autocorrelation

In [ ]:
#Splitting X and y and adding constant
y = df['mpg']
X = df.drop('mpg',axis = 1)
Xc = sm.add_constant(X)

In [ ]:
#Fitiing in model and applying autocorrleation
model = sm.OLS(y,Xc).fit()
import statsmodels.tsa.api as smt

acf = smt.graphics.plot_acf(model.resid,lags = 30,alpha = 0.05)

In [ ]:
model.summary()

Durbin-Watson statistic for any model will be between 0 and 4. To conclude that there is auto-correlation DW-statistic should be around 2. The DW statistic for the model is 2.028 and this indicate moderate level of autocorrelation.

# 4.Test of Linearity

In [ ]:
#Splitting X and y and adding constant
y = df['mpg']
X = df.drop('mpg',axis = 1)
Xc = sm.add_constant(X)

In [ ]:
#Fitting to model and applying linearity
model = sm.OLS(y,Xc).fit()
y_pred = model.predict(Xc)
resids = model.resid

sns.regplot(x = y_pred,y = resids,lowess=True,line_kws={'color':'red'})

In [ ]:
#Fitting to model and applying linearity
model = sm.OLS(y,Xc).fit()
y_pred = model.predict(Xc)
resids = model.resid

sns.regplot(x = y,y = y_pred,lowess=True,line_kws={'color':'red'})

The pattern expected when residuals are plotted against y_pred is a horizontal line around zero(0).
The pattern in the graph suggests that model design need to improve(we may need to add square and multiplicative terms to the model).

The expected pattern when y and y_pred is plotted is a line passing through origin with a slope of 45 degrees.

((#if model is perfect then price = price_predicted))

However the model shows a pattern away from expected.

In [ ]:
#Rainbow test for Linearity
import statsmodels.api as sm
sm.stats.diagnostic.linear_rainbow(res = model,frac = 0.5)

Ho: residuals exhibit linearity
H1: residuals exhibit non-linearity

Based on p-value , we say that residuals exhibhit linearity OR there is no non-linearity.

# Building Linear Regression Model

### Splitting data into training and testing sets

In [ ]:
#Applying in target variables
y = df['mpg']
X = df.drop('mpg',axis = 1)

In [ ]:
#Splitting the dataframe in 70:30(train and test)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=3)

In [ ]:
#printing the shape of train and test
print(X_train.shape,X_test.shape)

### Performing Linear Regression and calculating error terms

In [ ]:
#importing the library and building the model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error

lr = LinearRegression()
lr.fit(X_train,y_train)

y_test_pred = lr.predict(X_test)
y_train_pred = lr.predict(X_train)

print('r-square of Train :',r2_score(y_train,y_train_pred))
print('rmse for Train: ',np.sqrt(mean_squared_error(y_train,y_train_pred)))

print('\n')
print('r-square of Test :',r2_score(y_test,y_test_pred))
print('rmse for Test: ',np.sqrt(mean_squared_error(y_test,y_test_pred)))


There is a 4% difference in train and test.

# Feature Selection

# 1. Recursive Feature Elimination

In [ ]:
#importing libraries
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score,mean_squared_error

In [ ]:
#Fitting lr to rfe
lr = LinearRegression()
rfe = RFE(lr,n_features_to_select = 8)
rfe.fit(X,y)

In [ ]:
#Checking the rfe support function
rfe.support_

In [ ]:
pd.DataFrame(rfe.ranking_,index = X.columns,columns=['SELECT'])

This will give 8 best features with rank 1 and rest features with subsequent ranks.

In [ ]:
#Applying train test split method and splitting the data in 70:30 ratio
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=3)

In [ ]:
no_of_cols = X_train.shape[1]
r2score = []
rmse = []

lr = LinearRegression()  #estimator
for i in range(no_of_cols):
    rfe = RFE(lr,n_features_to_select=i+1)
    rfe.fit(X_train,y_train)
    y_test_pred = rfe.predict(X_test)
    
    #for r2score
    r2 = r2_score(y_test,y_test_pred)
    r2score.append(r2)
    
    #for rmse
    rms = np.sqrt(mean_squared_error(y_test,y_test_pred))
    rmse.append(rms)

In [ ]:
#plot for r2score
plt.plot(range(1,8),r2score)

In [ ]:
#printing r2score
r2score

In [ ]:
#plot for rmse
plt.plot(range(1,8),rmse)

In [ ]:
#printing rmse
rmse

# Hyperparamter Tuning to find the optimal number of features to keep

In [ ]:
from sklearn.model_selection import KFold,GridSearchCV

params = {'n_features_to_select':list(range(1,8))}
lr = LinearRegression()
rfe = RFE(lr)  #estimator

kf = KFold(n_splits=3,random_state=3)

gsearch = GridSearchCV(rfe,param_grid=params,scoring = 'r2',cv = kf,return_train_score=True)

In [ ]:
#to find the best parameter for our data
gsearch.get_params

# Forward selection to decide the best number of features to keep in the model(Step Forward Selection)


In [ ]:
#importing the SFS
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
sfs1 = sfs(lr,k_features=7,scoring='r2',cv = 3,verbose = 2)
sfs1.fit(X,y)

In [ ]:
#Making the dataframe as subset and transforming it
sf = pd.DataFrame(sfs1.subsets_).T
sf

In [ ]:
plt.plot(sf['avg_score'])

In [ ]:
cols = list(sfs1.k_feature_names_)
cols

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X[cols], y, test_size=0.3, random_state=3)

lr.fit(X_train, y_train)
y_test_pred = lr.predict(X_test)

plt.scatter(y_test, y_test_pred)
plt.plot(y_test, y_test,'r')

We can see that all the point are scattered close to the line.